# Min Headway
This workbook uses the GTFS data with some txt files with the routes of trains to find the min time between departures from a station.
To run the code you will need to install and extract the txt files from https://storage.googleapis.com/marduk-production/outbound/gtfs/rb_norway-aggregated-gtfs.zip and place them in Modeling\Vera\src\

In [ ]:
# Installing packages for jupyter lab
%pip install -r requirements.txt

In [65]:
import pandas as pd
import numpy as np

In [66]:
nodes_norway = pd.read_csv("../../Extraction/out/nodes_GTFS_ALL_FYLKER.csv")
edges_norway = pd.read_csv("../../Extraction/out/edges_GTFS_ALL_FYLKER_with_mondayTrips.csv")
display(nodes_norway.head()) 
display(edges_norway.head())

,id,stopPlaceId,name,lat,lon,modes,stopType
0,NSR:Quay:1173,NSR:Quay:1173,Oslo lufthavn,60.194220,11.099609,unknown,unknown
1,NSR:Quay:26169,NSR:Quay:26169,Sundvollen sentrum,60.062000,10.309736,bus,bus
2,NSR:Quay:26153,NSR:Quay:26153,Kroksund,60.066087,10.297885,bus,bus
3,NSR:Quay:26147,NSR:Quay:26147,Paddevika,60.074500,10.300500,bus,bus
4,NSR:Quay:26140,NSR:Quay:26140,Vik E16,60.077880,10.287076,"bus,coach service",multimodal


,from,to,lineId,lineCode,mode,authority,travelTimeSec,tripsInFeed,tripsOn2025_11_17
0,NSR:Quay:1173,NSR:Quay:1173,AVI:Line:WF_OSL-OSL,Oslo-Oslo,unknown,Avinor,11400.0,5,2
1,NSR:Quay:26169,NSR:Quay:26153,BRA:Line:26_4260,Flybuss Ringerike - Gardermoen,bus,Askeladden Reiser,NaN,6,8
2,NSR:Quay:26153,NSR:Quay:26147,BRA:Line:26_4260,Flybuss Ringerike - Gardermoen,bus,Askeladden Reiser,60.0,6,8
3,NSR:Quay:26147,NSR:Quay:26140,BRA:Line:26_4260,Flybuss Ringerike - Gardermoen,bus,Askeladden Reiser,60.0,6,8
4,NSR:Quay:26140,NSR:Quay:26116,BRA:Line:26_4260,Flybuss Ringerike - Gardermoen,bus,Askeladden Reiser,60.0,6,8


In [67]:
edges_norway = edges_norway[edges_norway["mode"] == "rail"]
nodes_norway = nodes_norway[nodes_norway["modes"].str.contains("rail")]
display(edges_norway.head())
display(nodes_norway.head())
print(f"Edges shape Oslo: {edges_norway.shape} | Nodes/Stops shape: {nodes_norway.shape}")

,from,to,lineId,lineCode,mode,authority,travelTimeSec,tripsInFeed,tripsOn2025_11_17
9085,NSR:Quay:1150,NSR:Quay:247,FLT:Line:FLY2,FLY2,rail,Flytoget,240.0,2940,100
9086,NSR:Quay:247,NSR:Quay:236,FLT:Line:FLY2,FLY2,rail,Flytoget,180.0,2940,100
9087,NSR:Quay:236,NSR:Quay:476,FLT:Line:FLY2,FLY2,rail,Flytoget,240.0,2940,100
9088,NSR:Quay:476,NSR:Quay:557,FLT:Line:FLY2,FLY2,rail,Flytoget,360.0,2940,100
9089,NSR:Quay:557,NSR:Quay:447,FLT:Line:FLY2,FLY2,rail,Flytoget,1140.0,6042,100


,id,stopPlaceId,name,lat,lon,modes,stopType
535,NSR:Quay:30086,NSR:Quay:30086,Gulskogen gård,59.746154,10.163695,"bus,rail",multimodal
555,NSR:Quay:28541,NSR:Quay:28541,Mjøndalen stasjon,59.750479,10.012134,"bus,rail",multimodal
574,NSR:Quay:30087,NSR:Quay:30087,Gulskogen gård,59.745948,10.163864,"bus,rail",multimodal
579,NSR:Quay:28539,NSR:Quay:28539,Mjøndalen stasjon,59.750649,10.011621,"bus,rail",multimodal
619,NSR:Quay:26294,NSR:Quay:26294,Hokksund stasjon,59.767310,9.911250,"bus,rail",multimodal


Edges shape Oslo: (1156, 9) | Nodes/Stops shape: (349, 7)


In [68]:
stop_dict = {}

for _, row in nodes_norway.iterrows():
    stop_id = row["stopPlaceId"]
    name = row["name"]
    stop_dict[stop_id] = name

In [69]:
edges_norway = edges_norway.rename(columns={"from": "from_stop"})
edges_norway = edges_norway.rename(columns={"to": "to_stop"})

for row in edges_norway.itertuples():
    edges_norway.loc[row.Index, "from_stop"] = stop_dict[row.from_stop]
    edges_norway.loc[row.Index, "to_stop"] = stop_dict[row.to_stop]

In [70]:
display(edges_norway.head())

,from_stop,to_stop,lineId,lineCode,mode,authority,travelTimeSec,tripsInFeed,tripsOn2025_11_17
9085,Stabekk stasjon,Lysaker stasjon,FLT:Line:FLY2,FLY2,rail,Flytoget,240.0,2940,100
9086,Lysaker stasjon,Skøyen stasjon,FLT:Line:FLY2,FLY2,rail,Flytoget,180.0,2940,100
9087,Skøyen stasjon,Nationaltheatret stasjon,FLT:Line:FLY2,FLY2,rail,Flytoget,240.0,2940,100
9088,Nationaltheatret stasjon,Oslo S,FLT:Line:FLY2,FLY2,rail,Flytoget,360.0,2940,100
9089,Oslo S,Oslo lufthavn stasjon,FLT:Line:FLY2,FLY2,rail,Flytoget,1140.0,6042,100


In [71]:
stop_times = pd.read_csv("src/stop_times.txt", dtype={5: str})
calender_dates = pd.read_csv("src/calendar_dates.txt")
display(calender_dates.head())
display(stop_times.head())

,service_id,date,exception_type
0,AKT:DayType:100_1,20251222,2
1,AKT:DayType:100_1,20251223,2
2,AKT:DayType:100_1,20251224,2
3,AKT:DayType:100_1,20251225,2
4,AKT:DayType:100_1,20251226,2


,trip_id,stop_id,arrival_time,departure_time,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled
0,AKT:ServiceJourney:11885_32_25_63000_64500_393...,NSR:Quay:44659,08:45:00,08:45:00,1,NaN,NaN,1.0,0.0
1,AKT:ServiceJourney:11885_32_25_63000_64500_393...,NSR:Quay:38208,08:47:00,08:47:00,2,NaN,NaN,NaN,1918.0
2,AKT:ServiceJourney:11885_32_25_63000_64500_393...,NSR:Quay:41859,08:48:00,08:48:00,3,NaN,NaN,NaN,2797.0
3,AKT:ServiceJourney:11885_32_25_63000_64500_393...,NSR:Quay:41904,08:48:00,08:48:00,4,NaN,NaN,NaN,3093.0
4,AKT:ServiceJourney:11885_32_25_63000_64500_393...,NSR:Quay:44711,08:49:00,08:49:00,5,NaN,NaN,NaN,4743.0


In [72]:
calender_dates = calender_dates[calender_dates["date"] == 20251201]
monday_service = calender_dates["service_id"].tolist()
print(monday_service)

['AKT:DayType:28_1', 'AKT:DayType:91_1', 'ATB:DayType:595', 'ATU:DayType:3ba7a127-0217-4f8e-822a-b6fd3fd38f86-3efce003-d2b0-48e9-94f1-c5e3113469dd-4c76e305-0857-4899-8092-b833c0a690b5-9f29b6f5-6da9-4abb-8b2a-12a49d1a6acb-e90d2816-6155-45e3-98ac-3308574c1e99-f1db2e39-17c5-45ce-b2b4-9bafc5491469-f78107f2-9db827663487', 'ATU:DayType:3ba7a127-0217-4f8e-822a-b6fd3fd38f86-4c76e305-0857-4899-8092-b833c0a690b5-9f29b6f5-6da9-4abb-8b2a-12a49d1a6acb-e90d2816-6155-45e3-98ac-3308574c1e99-f1db2e39-17c5-45ce-b2b4-9bafc5491469-f78107f2-9db2-4325-b71b-e3fb226e01c1', 'ATU:DayType:3ba7a127-0217-4f8e-822a-b6fd3fd38f86-4c76e305-0857-4899-8092-b833c0a690b5-9f29b6f5-6da9-4abb-8b2a-12a49d1a6acb-e90d2816-6155-45e3-98ac-3308574c1e99-f78107f2-9db2-4325-b71b-e3fb226e01c1', 'ATU:DayType:3ba7a127-0217-4f8e-822a-b6fd3fd38f86-9f29b6f5-6da9-4abb-8b2a-12a49d1a6acb-e90d2816-6155-45e3-98ac-3308574c1e99', 'ATU:DayType:3efce003-d2b0-48e9-94f1-c5e3113469dd-9f29b6f5-6da9-4abb-8b2a-12a49d1a6acb', 'ATU:DayType:9f29b6f5-6da9-4a

In [73]:
trips = pd.read_csv("src/trips.txt")
trips = trips[trips["service_id"].isin(monday_service)]
display(trips.head())
monday_trips = trips["trip_id"].tolist()

,route_id,trip_id,service_id,trip_headsign,direction_id,shape_id,wheelchair_accessible
1099,AKT:Line:105,AKT:ServiceJourney:15359_11_28_49800_51060_400...,AKT:DayType:28_1,Froland (Ovelandsheia) via Myra,0,AKT:JourneyPattern:81049_1,NaN
1101,AKT:Line:105,AKT:ServiceJourney:15361_21_28_53400_54660_400...,AKT:DayType:28_1,Froland (Ovelandsheia) via Myra,0,AKT:JourneyPattern:81049_1,NaN
1103,AKT:Line:105,AKT:ServiceJourney:15363_31_22_57000_58500_400...,AKT:DayType:28_1,Myra,0,AKT:JourneyPattern:83106_1,NaN
1107,AKT:Line:105,AKT:ServiceJourney:15367_51_22_64200_65700_400...,AKT:DayType:28_1,Myra,0,AKT:JourneyPattern:83106_1,NaN
1109,AKT:Line:105,AKT:ServiceJourney:15369_61_28_67800_69060_400...,AKT:DayType:28_1,Froland (Ovelandsheia) via Myra,0,AKT:JourneyPattern:81049_1,NaN


In [74]:
stop_times = stop_times[stop_times["trip_id"].isin(monday_trips)]
stop_times_by_stop = stop_times.sort_values(by=['stop_id', 'departure_time'])
display(stop_times_by_stop.head())

,trip_id,stop_id,arrival_time,departure_time,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled
8421336,VYG:ServiceJourney:308-DOM_349009-R,NSR:Quay:10,05:54:00,05:54:00,3,NaN,NaN,NaN,45775.0
4329099,SJN:ServiceJourney:40-TND_17664-R,NSR:Quay:10,09:16:00,09:18:00,10,NaN,NaN,NaN,255602.0
4330334,SJN:ServiceJourney:43-LOD_22256-R,NSR:Quay:10,11:25:00,11:29:00,10,NaN,NaN,NaN,294888.0
4331270,SJN:ServiceJourney:44-TND_17222-R,NSR:Quay:10,13:13:00,13:15:00,9,NaN,NaN,NaN,255602.0
4332147,SJN:ServiceJourney:45-LOD_17224-R,NSR:Quay:10,13:32:00,13:34:00,11,NaN,NaN,NaN,294887.0


In [75]:
stop_times_by_stop['departure_time_dt'] = pd.to_timedelta(stop_times_by_stop['departure_time'])
min_gaps = {}
for stop, group in stop_times_by_stop.groupby('stop_id'):
    
    departures = group['departure_time_dt'].sort_values()
    
    diffs = departures.diff().dropna()
    
    if not diffs.empty:
        min_gap = diffs.min()
        min_gaps[stop] = min_gap
    else:
        min_gaps[stop] = pd.Timedelta(0)

print(min_gaps)

{'NSR:Quay:10': Timedelta('0 days 00:19:00'), 'NSR:Quay:100': Timedelta('0 days 00:36:00'), 'NSR:Quay:10000': Timedelta('0 days 00:14:00'), 'NSR:Quay:10001': Timedelta('0 days 00:00:00'), 'NSR:Quay:100017': Timedelta('0 days 00:02:00'), 'NSR:Quay:100018': Timedelta('0 days 00:15:00'), 'NSR:Quay:10002': Timedelta('0 days 00:28:00'), 'NSR:Quay:100021': Timedelta('0 days 00:00:00'), 'NSR:Quay:100022': Timedelta('0 days 06:25:00'), 'NSR:Quay:100023': Timedelta('0 days 06:23:00'), 'NSR:Quay:100024': Timedelta('0 days 06:31:00'), 'NSR:Quay:100025': Timedelta('0 days 00:00:00'), 'NSR:Quay:100026': Timedelta('0 days 00:01:00'), 'NSR:Quay:100027': Timedelta('0 days 00:01:00'), 'NSR:Quay:100028': Timedelta('0 days 00:00:00'), 'NSR:Quay:100029': Timedelta('0 days 00:00:00'), 'NSR:Quay:10003': Timedelta('0 days 00:00:00'), 'NSR:Quay:100030': Timedelta('0 days 00:00:00'), 'NSR:Quay:100031': Timedelta('0 days 00:23:00'), 'NSR:Quay:100032': Timedelta('0 days 02:27:00'), 'NSR:Quay:100033': Timedelta('

In [76]:
nodes_norway['min_headway'] = np.nan

for row in nodes_norway.itertuples():
    if row.stopPlaceId in min_gaps:
        nodes_norway.loc[row.Index, 'min_headway'] = min_gaps[row.stopPlaceId]

nodes_norway = nodes_norway.dropna(subset=['min_headway'])

display(nodes_norway.head())

C:\Users\Vegar\AppData\Local\Temp\ipykernel_4388\2133896094.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0 days 00:00:00' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  nodes_norway.loc[row.Index, 'min_headway'] = min_gaps[row.stopPlaceId]


,id,stopPlaceId,name,lat,lon,modes,stopType,min_headway
2280,NSR:Quay:7905,NSR:Quay:7905,Lysaker stasjon,59.912755,10.638489,"bus,rail,coach service",multimodal,0 days 00:00:00
2312,NSR:Quay:7263,NSR:Quay:7263,Sandvika bussterminal,59.893199,10.525278,"bus,rail,coach service",multimodal,0 days 00:00:00
4383,NSR:Quay:1150,NSR:Quay:1150,Stabekk stasjon,59.907994,10.607316,rail,rail,0 days 00:03:00
4384,NSR:Quay:247,NSR:Quay:247,Lysaker stasjon,59.913404,10.637244,rail,rail,0 days 00:02:00
4385,NSR:Quay:236,NSR:Quay:236,Skøyen stasjon,59.922268,10.678630,rail,rail,0 days 00:02:00
